### Simple example for performing symbolic regression for a set of points

In [1]:
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
from pathlib import Path
from functools import partial
import torch
from sympy import lambdify
import json

In [2]:
## Load equation configuration and architecture configuration
import omegaconf
with open('100M/eq_setting.json', 'r') as json_file:
  eq_setting = json.load(json_file)

cfg = omegaconf.OmegaConf.load("100M/config.yaml")

In [3]:
## Set up BFGS load rom the hydra config yaml
bfgs = BFGSParams(
        activated= cfg.inference.bfgs.activated,
        n_restarts=cfg.inference.bfgs.n_restarts,
        add_coefficients_if_not_existing=cfg.inference.bfgs.add_coefficients_if_not_existing,
        normalization_o=cfg.inference.bfgs.normalization_o,
        idx_remove=cfg.inference.bfgs.idx_remove,
        normalization_type=cfg.inference.bfgs.normalization_type,
        stop_time=cfg.inference.bfgs.stop_time,
    )


In [4]:
params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            )

In [5]:
weights_path = "../weights/100M.ckpt"

In [6]:
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()

/home/jbrugger/.virtualenvs/NeuralSymbolicRegressionThatScales_NewNew/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.3.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../weights/100M.ckpt`


In [7]:
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [8]:
# Create points from an equation
number_of_points = 500
n_variables = 1

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(eq_setting["total_variables"])))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "x_1*sin(x_1)" #Use x_1,x_2 and x_3 as independent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

In [9]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  torch.Size([500, 3])
y shape:  torch.Size([500])


In [10]:
output = fitfunc(X,y) 

/home/jbrugger/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,device=self.device).unsqueeze(0)
/home/jbrugger/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,device=self.device).unsqueeze(0)


Memory footprint of the encoder: 4.096e-05GB 


/home/jbrugger/.virtualenvs/NeuralSymbolicRegressionThatScales_NewNew/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...


In [11]:
output

{'all_bfgs_preds': ['((x_1)*(sin(x_1)))', '((x_1)*((cos(x_1))*(tan(x_1))))'],
 'all_bfgs_loss': [0.0, 5.1156677e-14],
 'best_bfgs_preds': ['((x_1)*(sin(x_1)))'],
 'best_bfgs_loss': [0.0]}

In [12]:
class Namespace():
    def __init__(self):
        pass

args = Namespace()
args.logging_level = 40
args.max_branching_factor = 2
args.max_depth_of_tree = 10
args.max_constants_in_tree = 5
args.number_equations = 10
args.max_num_nodes_in_syntax_tree = 30
args.num_calls_sampling = 10
args.sample_with_noise = False
args.how_to_select_node_to_delete = 'random'

args.precision = 'float32'

In [13]:
best_eqution = output['best_bfgs_preds'][0]
from src.equation_classes.infix_to_prefix import  InfixToPrefix
from src.syntax_tree.syntax_tree import SyntaxTree

tree = SyntaxTree(grammar=None, args=args)

obj = InfixToPrefix(possible_operator_2dim='/ * - + root'.split(),
                    possible_operator_1dim='ln sin'.split(),
                    possible_operands='')
for symbol in tree.operator_to_class.keys(): 
    best_eqution = best_eqution.replace(f'{symbol}', f' {symbol} ')
best_eqution = best_eqution.replace('(', ' ( ')
best_eqution = best_eqution.replace(')', ' ) ')
best_eqution = best_eqution.replace('x_1', ' x_1 ')
best_eqution = best_eqution.replace('x_0', ' x_0 ')
prefix = obj.infixToPrefix(best_eqution.split())
print(prefix)
tree.prefix_to_syntax_tree(prefix.split())

* x_1 sin x_1  


In [14]:
tree.print()

d: 0  id: 0     *
d: 1  id: 1.0   	x_1
d: 1  id: 512.0 	sin
d: 2  id: 513.0 		x_1


In [15]:
import pandas as pd 
df = pd.DataFrame(X.numpy(), columns=['x_0','x_1','x_2',])
df['y'] = y 
res = tree.residual(512, dataset=df)
res

FloatingPointError: divide by zero encountered in divide